In [1]:
%%init_spark
launcher.packages = ["graphframes:graphframes:0.8.1-spark3.0-s_2.12"]

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions._
import org.graphframes._
import org.apache.log4j.Logger
import org.apache.log4j.Level

Intitializing Scala interpreter ...

Spark Web UI available at http://10.91.67.150:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1610433271365)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions._
import org.graphframes._
import org.apache.log4j.Logger
import org.apache.log4j.Level


In [3]:
val spark = SparkSession.builder()
      .appName("GraphAnalysis").master("local[4]")
      .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3e05b954


In [4]:
val nodes_path = "/Users/LTV/Downloads/NitroShare/with_ast/common_nodes.parquet"
val edges_path = "/Users/LTV/Downloads/NitroShare/with_ast/common_edges.parquet"
val output_path = "/Users/LTV/Downloads/NitroShare/with_ast/largest_component"

nodes_path: String = /Users/LTV/Downloads/NitroShare/with_ast/common_nodes.parquet
edges_path: String = /Users/LTV/Downloads/NitroShare/with_ast/common_edges.parquet
output_path: String = /Users/LTV/Downloads/NitroShare/with_ast/largest_component


In [6]:
assert(nodes_path.endsWith(".parquet"), "Nodes should be stored in parquet format")
assert(edges_path.endsWith(".parquet"), "Edges should be stored in parquet format")

val nodes = spark.read.load(nodes_path)
val edges = spark.read.load(edges_path)
  .withColumnRenamed("source_node_id", "src")
  .withColumnRenamed("target_node_id", "dst")
  .withColumnRenamed("id", "rel_id")
  .withColumnRenamed("type", "rel_type")

println(s"Original graph")
println(s"Nodes: ${nodes.count()}, Edges: ${edges.count()}")

Original graph
Nodes: 1637944, Edges: 8687701


nodes: org.apache.spark.sql.DataFrame = [id: bigint, type: string ... 2 more fields]
edges: org.apache.spark.sql.DataFrame = [rel_id: bigint, rel_type: string ... 2 more fields]


In [8]:
println(s"Original graph")
println(s"Nodes: ${nodes.count()}, Edges: ${edges.count()}")
nodes.take(10).foreach(println)
edges.take(10).foreach(println)

Original graph
Nodes: 1637944, Edges: 8687701
[3,module,__future__,null]
[4,global_variable,__future__.absolute_import,null]
[5,global_variable,__future__.division,null]
[6,global_variable,__future__.print_function,null]
[7,module,sys,null]
[8,module,types,null]
[16,non_indexed_symbol,builtins,null]
[17,class,builtins.set,null]
[18,function,builtins.set.add,null]
[19,function,builtins.id,null]
[4,defines,3045236,3045237]
[6,defines,3045237,3045238]
[8,defines,3045238,3045239]
[10,imports,3045239,207]
[12,defines,3045239,3045240]
[14,defines,207,447]
[15,calls,3045239,447]
[17,uses,3045239,21]
[19,imports,3045239,208]
[21,imports,3045239,209]


In [9]:
val g = GraphFrame(nodes, edges).dropIsolatedVertices()

println(s"After dropping isolated")
println(s"Nodes: ${g.vertices.count()}, Edges: ${g.edges.count()}")

g.vertices.take(10).foreach(println)
g.edges.take(10).foreach(println)

After dropping isolated
Nodes: 1637944, Edges: 8687701
[474,non_indexed_symbol,builtins.BaseException,null]
[964,global_variable,typing.Union,null]
[1950,Op,Eq,188]
[5409,subword,PREFIX,1432]
[7747,subword,▁arg,2315]
[15057,subword,sep,361]
[15663,subword,083,7384]
[17499,subword,▁tf,7536]
[35484,non_indexed_symbol,pwd.struct_passwd.pw_name,null]
[51428,module,logging.handlers,null]
[4,defines,3045236,3045237]
[6,defines,3045237,3045238]
[8,defines,3045238,3045239]
[10,imports,3045239,207]
[12,defines,3045239,3045240]
[14,defines,207,447]
[15,calls,3045239,447]
[17,uses,3045239,21]
[19,imports,3045239,208]
[21,imports,3045239,209]


g: org.graphframes.GraphFrame = GraphFrame(v:[id: bigint, type: string ... 2 more fields], e:[src: bigint, dst: bigint ... 2 more fields])


In [10]:
sc.setCheckpointDir("temp")
val cc_result = g.connectedComponents.run()

cc_result: org.apache.spark.sql.DataFrame = [id: bigint, type: string ... 3 more fields]


In [11]:
val g_cc = GraphFrame(cc_result, edges)

g_cc: org.graphframes.GraphFrame = GraphFrame(v:[id: bigint, type: string ... 3 more fields], e:[src: bigint, dst: bigint ... 2 more fields])


In [12]:
println(s"In the largest component")
println(s"Nodes: ${g_cc.vertices.count()}, Edges: ${g_cc.edges.count()}")

In the largest component
Nodes: 1637944, Edges: 8687701


In [13]:
g_cc.vertices.take(10).foreach(println)
g_cc.edges.take(10).foreach(println)

[474,non_indexed_symbol,builtins.BaseException,null,3]
[964,global_variable,typing.Union,null,3]
[1950,Op,Eq,188,3]
[5409,subword,PREFIX,1432,3]
[7747,subword,▁arg,2315,3]
[15057,subword,sep,361,3]
[15663,subword,083,7384,3]
[17499,subword,▁tf,7536,3]
[35484,non_indexed_symbol,pwd.struct_passwd.pw_name,null,3]
[51428,module,logging.handlers,null,3]
[4,defines,3045236,3045237]
[6,defines,3045237,3045238]
[8,defines,3045238,3045239]
[10,imports,3045239,207]
[12,defines,3045239,3045240]
[14,defines,207,447]
[15,calls,3045239,447]
[17,uses,3045239,21]
[19,imports,3045239,208]
[21,imports,3045239,209]


In [14]:
val component_count = g_cc.vertices.groupBy("component").count().sort(desc("count"))

component_count: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [component: bigint, count: bigint]


In [18]:
val row = component_count.take(1)

row: Array[org.apache.spark.sql.Row] = Array([3,1637944])


In [25]:
row(0).getAs[Long]("component")

res15: Long = 3


In [26]:
row(0).getAs[Long]("count")

res16: Long = 1637944


In [27]:
val com_local = component_count.collect()

com_local: Array[org.apache.spark.sql.Row] = Array([3,1637944])
